In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import f1_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [11]:
def est_mc_baseline(y, labels, weights, metric, n=11):
    scores = []
    for _ in range(n):
        mc_yhat = np.random.choice(labels, size=y.shape[0], p = weights)
        scores.append(metric(y, mc_yhat))
    return np.array(scores)

rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.figo_num.value_counts()).reset_index()
        .rename(columns={"index": "label", "figo_num": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="weighted")

mc_baseline = est_mc_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="weighted"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.3665158371040725
Monte Carlo baseline: 0.36632086664743835


## SMBO

In [12]:
def objective(h_params, X, y, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = RandomForestClassifier(
        n_estimators=h_params[0],
        max_depth=h_params[1],
        max_features=h_params[2],
        min_samples_split=h_params[3],
        min_samples_leaf=h_params[4],
        bootstrap=h_params[5],
        n_jobs=-1,
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [13]:
space = [
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(int(10), int(100), name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(4), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf"),
    Categorical([True, False], name="bootstrap")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [14]:
scoring_default = "f1_weighted"
callback_file = f"{unified_dsets[dset_idx]}_opt_rfc_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[742, 49, 'log2', 3, 2, True]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.8023
Function value obtained: -0.3691
Current minimum: -0.3691
Iteration No: 2 started. Evaluating function at random point.
[470, 62, 'auto', 3, 2, False]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.8629
Function value obtained: -0.3887
Current minimum: -0.3887
Iteration No: 3 started. Evaluating function at random point.
[320, 73, 'log2', 3, 2, False]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.6539
Function value obtained: -0.3782
Current minimum: -0.3887
Iteration No: 4 started. Evaluating function at random point.
[741, 49, 'log2', 3, 2, False]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 3.0310
Function value obtained: -0.3685
Current minimum: -0.3887
Iteration No: 5 started. Evaluating function at random point.
[475, 69, 'log2', 2, 3, True]
Itera

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 2.1903
Function value obtained: -0.4064
Current minimum: -0.4188
Iteration No: 79 started. Searching for the next optimal point.
[137, 11, 'sqrt', 2, 3, False]
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 2.4554
Function value obtained: -0.3827
Current minimum: -0.4188
Iteration No: 80 started. Searching for the next optimal point.
[120, 98, 'sqrt', 4, 2, False]
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 2.4207
Function value obtained: -0.4047
Current minimum: -0.4188
Iteration No: 81 started. Searching for the next optimal point.
[954, 100, 'auto', 2, 1, False]
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 6.0033
Function value obtained: -0.3888
Current minimum: -0.4188
Iteration No: 82 started. Searching for the next optimal point.
[100, 100, 'auto', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 2.5751
Function value obtained: -0.4107
Current minimum: -0.4188
Iteration No: 83 started. Searching for the next optimal point.
[100, 100, 'auto', 2, 3, False]
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 2.2444
Function value obtained: -0.3855
Current minimum: -0.4188
Iteration No: 84 started. Searching for the next optimal point.
[122, 10, 'sqrt', 4, 1, True]
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 2.8123
Function value obtained: -0.3792
Current minimum: -0.4188
Iteration No: 85 started. Searching for the next optimal point.
[100, 100, 'auto', 2, 1, False]


...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 15.6457
Function value obtained: -0.4168
Current minimum: -0.4398
Iteration No: 299 started. Searching for the next optimal point.
[100, 10, 'sqrt', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 12.9125
Function value obtained: -0.4054
Current minimum: -0.4398
Iteration No: 300 started. Searching for the next optimal point.
[100, 10, 'sqrt', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 12.1789
Function value obtained: -0.4023
Current minimum: -0.4398
